In [ ]:
import torch
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from torch import nn

from tabpfn import TabPFNClassifier

X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

X_train = torch.from_numpy(X_train)
X_train.requires_grad = True
X_test = torch.from_numpy(X_test)
X_test.requires_grad = True
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

#given x,y,x_test,y_test with
#X_train.requires_grad = True
#X_test.requires_grad = True

# N_ensemble_configurations controls the number of model predictions that are ensembled with feature and class rotations (See our work for details).
# When N_ensemble_configurations > #features * #classes, no further averaging is applied.
classifier = TabPFNClassifier(device='cpu', N_ensemble_configurations=3, no_preprocess_mode=True, no_grad=False) # with no_grad=False, no_preprocessing=True, .... TODO:David
classifier.fit(X_train, y_train)

logits = classifier.predict_proba(X_test, return_logits=True) # would be awesome to allow returning logits instead of probabilities TODO:David


#loss = nn.CrossEntropyLoss()(probs, torch.from_numpy(y_test).long())
loss = nn.CrossEntropyLoss()(logits, y_test.long().to('cpu'))
loss.backward()

# now we should have a gradient on
(X_test.grad is not None) and (X_train.grad is not None)

print(X_test.grad.max(), X_train.grad.max())
#y_eval, p_eval = classifier.predict(X_test, return_winning_probability=True)

#print('Accuracy', accuracy_score(y_test, y_eval))

# Output cpu - 1 ensemble: tensor(3.9629, dtype=torch.float64) tensor(2.5293, dtype=torch.float64)
# Output cuda - 1 ensemble: tensor(3.9629, dtype=torch.float64) tensor(2.5293, dtype=torch.float64)
# Output cpu - 3 ensemble: tensor(3.4180, dtype=torch.float64) tensor(1.7904, dtype=torch.float64)
# Output cuda - 3 ensemble: tensor(3.4180, dtype=torch.float64) tensor(1.7904, dtype=torch.float64)

In [ ]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
output.backward()